In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
!pip install tensorflow_addons
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import os
import shutil

import tensorflow_hub as hub

!pip install tensorflow_text
import tensorflow_text as text
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')                                     

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 1.8 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 22.4 MB/s 
     |████████████████████████████████| 47.7 MB 80 kB/s 
     |████████████████████████████████| 90 kB 4.9 MB/s 
     |████████████████████████████████| 99 kB 6.0 MB/s 
     |████████████████████████████████| 43 kB 833 kB/s 
     |████████████████████████████████| 1.2 MB 33.9 MB/s 
     |████████████████████████████████| 352 kB 35.4 MB/s 
     |████████████████████████████████| 213 kB 32.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist

import nltk


nltk.download('punkt')

rewiews = pd.read_csv('/content/drive/MyDrive/hh/HeadHunter_train.csv', sep = ',',error_bad_lines = False,)
rewiews = rewiews.fillna('')
rewiews = rewiews.sample(frac=1)
#rewiews = rewiews[:5000]
#positives_data = rewiews['positive']
#negatives_data = rewiews['negative']
#all_words=' '.join(positives_data)
#all_words=all_words+' '+' '.join(negatives_data)
#all_words=word_tokenize(all_words)
#dist=FreqDist(all_words)
#num_unique_word=len(dist)
#num_unique_word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
rewiews

In [ ]:
rewiews['text'] = 'Позитив: '+rewiews['positive']+' Негатив: '+rewiews['negative']
rewiews

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,target,text
43398,164557,Северодвинск,Телемаркетолог,"Хороший коллектив (Как повезет), удобный графи...",Северодвинск. Ужасно неадекватное начальство ф...,3,5,1,2,2,3,0,"Позитив: Хороший коллектив (Как повезет), удоб..."
1845,7037,Москва,Продавец-кассир,Идея создателей бренда),Отношения к персоналу и заработную плату!,2,1,1,1,1,1,8,Позитив: Идея создателей бренда) Негатив: Отно...
7817,29643,Тверь,Логист,Всегда можно договориться об изменении рабочег...,"Если убрать небольшие задержки з п, все осталь...",4,5,5,3,5,4,0,Позитив: Всегда можно договориться об изменени...
22879,86956,Уфа,Оператор станков с ПУ,Ничего ничего ничего ничего,Все все все все все все все все,1,1,1,1,1,1,3,Позитив: Ничего ничего ничего ничего Негатив: ...
7920,30012,Казань,Подработка,Своевременная зарплата,Своевременная зарплата,5,5,5,5,5,5,8,Позитив: Своевременная зарплата Негатив: Своев...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22076,83764,Киров,Продавец-консультант,Близость к дому рабочего места,"Условия труда, распорядок дня, количество сотр...",3,3,4,3,3,3,8,Позитив: Близость к дому рабочего места Негати...
37051,140753,Серпухов,Старший менеджер по обслуживанию физических лиц,"Как мама 2х детей маленьких, которые зачастую ...","Зарплату. Пашем за 2х, а зп еще такие маленьки...",3,5,4,3,3,4,8,"Позитив: Как мама 2х детей маленьких, которые ..."
48082,182451,Саранск,Консультант,область профессиональной деятельности - консал...,"нормирование и оплата труда, нормирование и оп...",2,5,4,3,4,1,8,Позитив: область профессиональной деятельности...
39228,148956,Брянск,Тестировщик ПО,Слаженный и дружный коллектив,Продолжать совершенствоваться и никогда не ост...,5,5,5,5,5,5,8,Позитив: Слаженный и дружный коллектив Негатив...


In [ ]:
def target_list(ser):
  rez = np.zeros(9)
  for ind in ser.split(","):
    rez[int(ind)] = 1
  return rez
#target = rewiews['target'].apply(target_list)
#full_target = np.array(target.to_list())


In [ ]:
full_target.shape

(50876, 8)

In [ ]:
full_target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
num_classes = 8
def target_list(ser):
  rez = np.zeros(num_classes)
  for ind in ser.split(","):
    if ind!='0':
      rez[int(ind)-1] = 1
    #rez[int(ind)] = 1
  return rez
target = rewiews['target'].apply(target_list)
full_target = np.array(target.to_list())
full_target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
def target_list(ser):
 # print(ser)
  rez = []
  #rez[int(ser)] = 1
  for ind in ser.split(","):
    #rez[int(ind)] = 1/len(ser.split(","))
    rez+=[float(ind)]
   # print(1/len(ser.split(",")))
   # print(rez[int(ind)])
   #data_tensor = tf.ragged.constant(data)
  return np.array(rez)
#target = rewiews['target'].apply(target_list)
#full_target = target.to_list()
#full_target

In [ ]:
from keras.preprocessing.text import Tokenizer
#VOCAB_SIZE = 80000 
#tokenizer_main = Tokenizer(num_words=VOCAB_SIZE)
#tokenizer_main.fit_on_texts(list(positives_data)+list(negatives_data))
#Posi_train = tokenizer_main.texts_to_sequences(positives_data)
#Nega_train = tokenizer_main.texts_to_sequences(negatives_data)

In [ ]:
city_data = rewiews['city']
all_words=' '.join(city_data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
VOCAB_SIZE_CITIES =len(dist)

position_data = rewiews['position']
all_words=' '.join(position_data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
VOCAB_SIZE_POSITIONS =len(dist)

In [ ]:
tokenizer_city = Tokenizer(num_words=VOCAB_SIZE_CITIES)
tokenizer_city.fit_on_texts(list(city_data))
City_train = tokenizer_city.texts_to_sequences(city_data)


tokenizer_position = Tokenizer(num_words=VOCAB_SIZE_POSITIONS)
tokenizer_position.fit_on_texts(list(position_data))
Position_train = tokenizer_position.texts_to_sequences(position_data)

In [ ]:
from keras.preprocessing import sequence,text
MAX_SEQ_LEN = 100
#full_posi_train_data = sequence.pad_sequences(Posi_train, 
#                                    padding='post',
#                                    maxlen=MAX_SEQ_LEN)
#full_nega_train_data = sequence.pad_sequences(Nega_train, 
#                                    padding='post',
#                                    maxlen=MAX_SEQ_LEN)

full_city_train_data = sequence.pad_sequences(City_train, 
                                    padding='post',
                                    maxlen=5)
full_position_train_data = sequence.pad_sequences(Position_train, 
                                    padding='post',
                                    maxlen=35)

full_scores_train_data = rewiews[['salary_rating','team_rating','managment_rating','career_rating','workplace_rating','rest_recovery_rating']]
full_scores_train_data = np.array(full_scores_train_data)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 129
seed = 42
dropout_rate = 0.1
#full_posi_train_data = tf.data.Dataset.from_tensor_slices(((rewiews['positive'],rewiews['negative']),full_target))
#full_nega_train_data = tf.data.Dataset.from_tensor_slices(rewiews['negative'])
#target = tf.data.Dataset.from_tensor_slices(full_target)
#{'image_input': x_train, 'label_input': y_train},
full_ds = tf.data.Dataset.from_tensor_slices(({'text':rewiews['text'],'cities':full_city_train_data,'positions':full_position_train_data,'scores':full_scores_train_data},full_target))
full_ds = full_ds.shuffle(60000, seed=seed)
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
train_ds = full_ds.take(50000).batch(BATCH_SIZE)
val_ds = full_ds.skip(50000).batch(BATCH_SIZE)

In [ ]:
train_ds

<BatchDataset shapes: ({text: (None,), cities: (None, 5), positions: (None, 35), scores: (None, 6)}, (None, 8)), types: ({text: tf.string, cities: tf.int32, positions: tf.int32, scores: tf.int64}, tf.float64)>

In [ ]:

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
tfhub_handle_encoder =  'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_model_name = 'small_bert/bert_en_uncased_L-8_H-256_A-4'
tfhub_handle_encoder =  'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/2'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_model_name = 'bert_multi_cased_L-12_H-768_A-12'
tfhub_handle_encoder =  'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3'
#bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
#bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='positives')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['sequence_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  #net = tf.keras.layers.Dense(768, activation='sigmoid')(net)
  net = tf.keras.layers.Flatten()(net)
  net = tf.keras.layers.Dense(9, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)
model = build_classifier_model()

In [ ]:
tfhub_handle_encoder

'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/2'

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["Недавно устроился в ****** на должность кладовщика. Коллектив дружный и приветливый к новичкам - во всем помогают и поддерживают."])
print(embedding_model(sentences))

In [ ]:
class Cities(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE_CITIES,8,input_length = 5)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=dropout_rate, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        out = self.embedding(inp)
        out = self.lstm(out)
        return out

class Positions(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE_POSITIONS,16,input_length = 35,)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=dropout_rate, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        #print('Positions',inp)
        out = self.embedding(inp)
        #print('Positions',out)
        out = self.lstm(out)
        return out

class Positives(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='positives')
        self.preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='Positives_preprocessing')

    def call(self, inp, is_training=False):
        #print('Positives inp',inp)
        out = self.text_input(inp)
        return out

class Negatives(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='positives')
        self.preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='Negatives_preprocessing')

    def call(self, inp, is_training=False):
        out = self.text_input(inp)
        return out

class Bert(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='text_preprocessing')
        self.encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='text_BERT_encoder')
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, inp, is_training=False):
        out = self.preprocessing_layer(inp)
        out = self.encoder(out)
        out = self.dropout(out['sequence_output'])
        out = self.flatten(out)
        return out

class Scores(tf.keras.Model):
    def __init__(self):
        #print('Scores init')
        super().__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(7,1))
        self.dense1 = tf.keras.layers.Dense(256,activation='sigmoid')
        self.dense2 = tf.keras.layers.Dense(128,activation='sigmoid')
        self.dense3 = tf.keras.layers.Dense(32,activation='sigmoid')
        self.drop1 = tf.keras.layers.Dropout(dropout_rate)
        self.drop2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inp, is_training=False):
        #print('Scores call, input: ',inp)
        out = self.input_layer(inp)
        #print('Scores call, out: ',out)
        
        out = self.dense1(out)
        out = self.drop1(out)
        out = self.dense2(out)
        out = self.drop2(out)
        out = self.dense3(out)
        return out

class FiveHeadNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.posi = Positives()
        self.nega = Negatives()
        self.bert = Bert()
        self.scor = Scores()
        self.city = Cities()
        self.pos = Positions()
        #self.dense = tf.keras.layers.Dense(160,activation=None)
        self.out = tf.keras.layers.Dense(num_classes,activation=None, name='classifier')

    def call(self, inp):

        out_bert = self.bert(inp['text'])
        out_pos = self.pos(inp['positions'])
        out_s = self.scor(inp['scores'])
        out_c = self.city(inp['cities'])
        out = tf.concat([out_bert, out_s,out_c,out_pos], axis=1)
        #out = tf.concat([out_p, out_n, out_s], axis=1)
        #out = self.dense(out)
        out = self.out(out)
        #print('out')
        return out

metric = tfa.metrics.FBetaScore(num_classes=9)

model = FiveHeadNet()


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train[4]

9

In [ ]:
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

#metrics = tf.metrics.BinaryAccuracy()
metric = tfa.metrics.FBetaScore(num_classes=num_classes)


In [ ]:
epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
optimizer=tf.keras.optimizers.Adam(learning_rate=init_lr)

In [ ]:
model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=(metric,'accuracy'))

In [ ]:
for text_batch, label_batch in val_ds.take(1):
  print(text_batch['negatives'][0])
  print(f'Label : {label_batch[0]}')

tf.Tensor(b'\xd0\x9d\xd0\xb8\xd1\x87\xd0\xb5\xd0\xb3\xd0\xbe \xd1\x85\xd0\xbe\xd1\x80\xd0\xbe\xd1\x88\xd0\xb5\xd0\xb3\xd0\xbe \xd1\x82\xd0\xb0\xd0\xbc \xd0\xbd\xd0\xb5 \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xbe', shape=(), dtype=string)
Label : [0. 0. 0. 0. 0. 0. 0. 1.]


In [ ]:
model.load_weights('/content/drive/MyDrive/hh/model.weights')

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
last_epoch = 1
for i in range(last_epoch,last_epoch+10):
  print('epoch',last_epoch)
  history = model.fit(#rewiews['positive'],#
                    train_ds,
                    validation_data = val_ds,
                    #full_target,
                    #validation_data = ([posi_val_data,nega_val_data,scores_val_data,city_val_data,position_val_data],target_val),
                    epochs=1,
                    #batch_size=BATCH_SIZE,
                    #verbose=1,
                    #validation_split=0.15,
                    #steps_per_epoch=20
                    )
  model.save_weights('/content/drive/MyDrive/hh/model.weights')
#loss: 0.0536 - fbeta_score: 0.8603 - accuracy: 0.8933
#loss: 0.0480 - fbeta_score: 0.8869 - accuracy: 0.9054
#loss: 0.0455 - fbeta_score: 0.8933 - accuracy: 0.9102
#loss: 0.0404 - fbeta_score: 0.9028 - accuracy: 0.9211
#loss: 0.0401 - fbeta_score: 0.9133 - accuracy: 0.9226
#388/388 [==============================] - 2279s 6s/step - loss: 0.0707 - fbeta_score: 0.4478 - accuracy: 0.5478 - val_loss: 0.0606 - val_fbeta_score: 0.4368 - val_accuracy: 0.5217
#epoch 7 388/388 [==============================] - 1249s 3s/step - loss: 0.0619 - fbeta_score: 0.5408 - accuracy: 0.5703 - val_loss: 0.0589 - val_fbeta_score: 0.5358 - val_accuracy: 0.5696


Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4
372/388 [===========================>..] - ETA: 50s - loss: 0.0590 - fbeta_score: 0.5697 - accuracy: 0.5709

In [ ]:
model.save_weights('/content/model.weights')

In [ ]:
pred = model.predict(rewiews['positive'][:7])

In [ ]:
pred

array([[5.34477413e-01, 5.18208928e-02, 6.81548670e-04, 2.86875671e-04,
        3.22093000e-03, 1.78014545e-03, 3.97835486e-02, 1.42840832e-03,
        4.73960549e-01],
       [4.05141026e-01, 1.29202362e-02, 1.32556423e-04, 2.28116708e-03,
        1.02339662e-04, 5.84402296e-04, 1.24074733e-02, 6.68809749e-04,
        7.34329820e-01],
       [2.65102714e-01, 5.13456129e-02, 3.09774769e-04, 3.03705386e-03,
        4.73944878e-04, 1.48293702e-03, 1.39532359e-02, 2.70808791e-03,
        7.70116031e-01],
       [2.13273212e-01, 1.69029515e-02, 6.87537889e-04, 2.77626747e-03,
        4.06718289e-04, 1.57282525e-03, 4.31789607e-02, 3.45434900e-03,
        7.83573210e-01],
       [8.72026384e-01, 4.08108570e-02, 9.24536493e-03, 1.91258179e-04,
        5.13936952e-03, 2.83363331e-02, 1.05112568e-02, 2.17615953e-03,
        5.48029393e-02],
       [1.00059316e-01, 1.23157771e-02, 5.41880145e-04, 1.65069755e-03,
        4.50457679e-04, 1.66710909e-03, 1.50438296e-02, 2.26140791e-03,
        8.9

In [ ]:
model.load_weights('/content/model.weights')

In [ ]:
for text_batch in test_ds.take(1):
  print(text_batch['negatives'])

In [ ]:
pred_test = model.predict(test_ds)

Positives inp Tensor("IteratorGetNext:3", shape=(None,), dtype=string)
Positives out Tensor("five_head_net/positives/flatten/Reshape:0", shape=(None, 98304), dtype=float32)


In [ ]:
def tostr(lis):
  ret = []
  for i in range(9):
    if lis[i]>=0.5:
      ret+=[i]
  if len(ret) == 0:
    return(str(np.argmax(lis)))
    #return '0'
  else:
    return ','.join(map(str, ret))
tostr(pred_test[33])

'8'

In [ ]:
formatted_predictions = []
pred_test_copy = pred_test.copy()
for prediction in pred_test_copy:
  #print(prediction)
  #print(tostr(prediction))
  formatted_predictions+=[tostr(prediction)]

In [ ]:
final = pd.DataFrame(formatted_predictions)
final['review_id'] = test_rewiews['review_id']
final.columns = ['target','review_id']

final[['review_id','target']].to_csv('/content/final.csv', index=False)#, escapechar = '"',quoting=csv.QUOTE_MINIMAL)

In [ ]:
final

,target,review_id
0,8,3
1,8,4
2,8,13
3,0,14
4,8,17
...,...,...
50646,0,192819
50647,"0,8",192821
50648,"1,8",192830
50649,"1,8",192832
